<a href="https://colab.research.google.com/github/NicholasKTaylor/Coursera_Capstone/blob/master/Week4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# England And Wales Housing Affordability Versus Leisure Opportunities 
## Set up libraries

In [65]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Libraries imported.


## The UK Office for National Statistices publish a set of Affordability ratios calculated by dividing house prices by gross annual residence-based earnings. Based on the median and lower quartiles of both house prices and earnings in England and Wales.  We will use this dataset.  Sheet 3c contains the ratio of median house price to median gross annual earnings organised by English County, for years 2002 to 2018.  We will use the data for 2018.  We also need geospatial coordinates for each County.

In [66]:
link='https://github.com/NicholasKTaylor/Coursera_Capstone/blob/master/ratioofhousepricetoresidencebasedearningslowerquartileandmedian.csv'
#'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/housing/datasets/ratioofhousepricetoresidencebasedearningslowerquartileandmedian/current/ratioofhousepricetoresidencebasedearningslowerquartileandmedian.xls'

# Read_excel and read_csv dont work with a link to a file in Colab, which is a pain.  Need to read as html and get rid of spurious first column
tables=pd.read_html(link)
df=tables[0].reset_index(drop=True)
#df.drop([df.columns[0]],axis=1,inplace=True)
# Remove redundant columns 
newIndex=['Name','2018']
df2 = df.reindex(newIndex,axis=1)
df2.head()




,Name,2018
0,Buckinghamshire,11.06
1,Cambridgeshire,9.02
2,Cumbria,5.62
3,Derbyshire,6.04
4,Devon,9.42


In [67]:
#  Search GEO data and add to our dataframe

geolocator = Nominatim(user_agent="to_explorer")
lat=[]
lon=[]
#lat_lon.head()
for name in df2['Name'] :
  address = name + ', UK'
  location = geolocator.geocode(address)
  lat.append(location.latitude)
  lon.append(location.longitude)
  #lat_lon.append(pd.Series([name,longitude,latitude]),ignore_index=True)
 # print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

df2['Longitude']=lon
df2['Latitude']=lat

df2.head()

,Name,2018,Longitude,Latitude
0,Buckinghamshire,11.06,-0.750000,51.750000
1,Cambridgeshire,9.02,0.083333,52.333333
2,Cumbria,5.62,-2.942098,54.614314
3,Derbyshire,6.04,-1.556618,53.118503
4,Devon,9.42,-3.750000,50.750000


### Get The Location of somewhere pretty central

In [68]:
address = 'Northampton, UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Northampton are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Northampton are 52.2381355, -0.8963907.


In [64]:
# create map of England Centred On Northampton
# Zoom level of 7 is appropriate
map_Eng = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Eng)  
    
map_Eng

### Use Foursquare API to explore the Counties and segment them

In [69]:
# Lets just look at the first County:
county_latitude = df2.loc[0, 'Latitude'] # county latitude value
county_longitude = df2.loc[0, 'Longitude'] # county longitude value

county_name = df2.loc[0, 'Name'] # county name

print('Latitude and longitude values of {} are {}, {}.'.format(county_name, 
                                                               county_latitude, 
                                                               county_longitude))

CLIENT_ID = 'YI5XJRJ2TC1ELJE333WZYJILHOFEKJFR3T0BPKHBJ2SZL5YT' # your Foursquare ID
CLIENT_SECRET = 'PXSJSS1OTE4XZ4TLFZEL20HR50ES2DCVIDISS5AN0IK24RU2' # your Foursquare Secret
VERSION = '20191021' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 35000 # define radius 35km
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    county_latitude, 
    county_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of Buckinghamshire are 51.75, -0.75.


{'meta': {'code': 200, 'requestId': '5dadd8dfa87921003942b9c0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b793ec3f964a52092f02ee3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '4b793ec3f964a52092f02ee3',
       'location': {'address': '26 High Street',
        'cc': 'GB',
        'city': 'Wendover',
        'country': 'United Kingdom',
        'distance': 1441,
        'formattedAddress': ['26 High Street',
         'Wendover',
         'Buckinghamshire',
         'HP22 6EA',
         'United Kingdom'],
        'labeledLatLng

### Borrow getNearbyVenues function from the labs 

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['County', 
                  'County Latitude', 
                  'County Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Find nearby venues for England

In [72]:
# Extract all the nearby venues for the England County dataset
# Use radius of 35km
county_venues = getNearbyVenues(names=df2['Name'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude'],
                                   radius=35000
                                  )
# Check the vebue data returned
print(county_venues.shape)
county_venues.head()

# Let's check how many venues were returned for each neighborhood
county_venues.groupby('County').count()

# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(county_venues['Venue Category'].unique())))





Buckinghamshire
Cambridgeshire
Cumbria
Derbyshire
Devon
Dorset
East Sussex
Essex
Gloucestershire
Hampshire
Hertfordshire
Kent
Lancashire
Leicestershire
Lincolnshire
Norfolk
Northamptonshire
North Yorkshire
Nottinghamshire
Oxfordshire
Somerset
Staffordshire
Suffolk
Surrey
Warwickshire
West Sussex
Worcestershire
(2700, 7)
There are 212 uniques categories.


In [74]:
county_venues.tail()

,County,County Latitude,County Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2695,Worcestershire,52.210949,-2.156956,Hyatt Regency Birmingham,52.478096,-1.909324,Hotel
2696,Worcestershire,52.210949,-2.156956,Packwood House,52.348943,-1.745753,Monument / Landmark
2697,Worcestershire,52.210949,-2.156956,DoubleTree by Hilton Stratford-upon-Avon,52.193979,-1.713047,Hotel
2698,Worcestershire,52.210949,-2.156956,Krispy Kreme,52.478034,-1.892792,Donut Shop
2699,Worcestershire,52.210949,-2.156956,Rajdoot Tandoori,52.482635,-1.910547,Indian Restaurant


### Analyze Each County

In [75]:
# one hot encoding
county_onehot = pd.get_dummies(county_venues[['Venue Category']], prefix="", prefix_sep="")

# add county column back to dataframe
county_onehot['County'] = county_venues['County'] 

# move neighborhood column to the first column
fixed_columns = [county_onehot.columns[-1]] + list(county_onehot.columns[:-1])
county_onehot = county_onehot[fixed_columns]

county_onehot.head()
county_onehot.shape

# Group rows by county
county_grouped = county_onehot.groupby('County').mean().reset_index()
county_grouped
county_grouped.shape

# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for county in county_grouped['County']:
    print("----"+county+"----")
    temp = county_grouped[county_grouped['County'] == county].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
    

----Buckinghamshire----
                venue  freq
0                 Pub  0.17
1               Hotel  0.08
2           Gastropub  0.07
3  Italian Restaurant  0.04
4         Coffee Shop  0.04


----Cambridgeshire----
          venue  freq
0           Pub  0.15
1           Bar  0.07
2          Café  0.07
3   Coffee Shop  0.06
4  Burger Joint  0.05


----Cumbria----
        venue  freq
0         Pub  0.16
1       Hotel  0.16
2        Café  0.08
3  Restaurant  0.06
4         Bar  0.05


----Derbyshire----
                          venue  freq
0                           Pub  0.23
1                          Park  0.09
2                   Coffee Shop  0.06
3                          Café  0.04
4  Theme Park Ride / Attraction  0.04


----Devon----
         venue  freq
0          Pub  0.20
1         Café  0.08
2        Hotel  0.06
3          Bar  0.05
4  Coffee Shop  0.05


----Dorset----
            venue  freq
0             Pub  0.16
1            Café  0.08
2           Beach  0.07
3        

In [78]:
county_grouped.head()

,County,African Restaurant,Airport,Airport Lounge,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Café,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Library,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hill,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Library,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,National Park,Nature Preserve,Nightclub,Noodle House,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Racecourse,Racetrack,Record Shop,Recreation Center,Reservoir,Resort,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Rugby Stadium,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoothie Shop,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Track Stadium,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vineyard,Warehouse Store,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yakitori Restaurant,Yoga Studio,Zoo
0,Buckinghamshire,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.01,0.00,0.0,0.00,0.0,0.01,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.01,0.03,0.0,0.0,0.00,0.00,0.04,0.0,0.0,0.02,0.0,0.0,0.02,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.00,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.01,0.03,0.0,0.01,0.0,0.01,0.01,0.0,0.07,0.01,0.0,0.00,0.01,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.02,0.02,0.08,0.01,0.00,0.02,0.01,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.00,0.01,0.00,0.00,0.0,0.0,0.00,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.04,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.01,0.17,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.02,0.00,0.0,0.00,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.01,0.01,0.00,0.0,0.00,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.01
1,Cambridgeshire,0.01,0.0,0.0,0.01,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.03,0.07,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.03,0.01,0.0,0.0,0.0,0.02,0.01,0.0,0.0,0.05,0.00,0.07,0.0,0.0,0.00,

### Put data into a dataframe

In [0]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
#Create dataframe and display top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['County']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
counties_venues_sorted = pd.DataFrame(columns=columns)
counties_venues_sorted['County'] = county_grouped['County']

for ind in np.arange(county_grouped.shape[0]):
    counties_venues_sorted.iloc[ind, 1:] = return_most_common_venues(county_grouped.iloc[ind, :], num_top_venues)

counties_venues_sorted.head()

,County,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buckinghamshire,Pub,Hotel,Gastropub,Italian Restaurant,Coffee Shop,Park,Café,Brewery,French Restaurant,Canal Lock
1,Cambridgeshire,Pub,Café,Bar,Coffee Shop,Burger Joint,Park,Gastropub,Restaurant,Bookstore,Tea Room
2,Cumbria,Pub,Hotel,Café,Restaurant,Bar,Coffee Shop,Gastropub,Supermarket,Scenic Lookout,Italian Restaurant
3,Derbyshire,Pub,Park,Coffee Shop,Theme Park Ride / Attraction,Café,Scenic Lookout,Historic Site,Theater,Gastropub,Food & Drink Shop
4,Devon,Pub,Café,Hotel,Bar,Gastropub,Coffee Shop,Beach,Italian Restaurant,Trail,Restaurant


### Cluster the neighbourhoods


In [92]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

county_grouped_clustering = county_grouped.drop('County', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(county_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each county.

# add clustering labels
counties_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

county_merged = df2
county_merged.rename(columns = {'Name':'County','2018':'Affordability'}, inplace = True)


# merge county_grouped with df2 to add latitude/longitude for each neighborhood
county_merged = county_merged.join(counties_venues_sorted.set_index('County'), on='County')

county_merged.head() # check the last columns!


,County,Affordability,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buckinghamshire,11.06,-0.750000,51.750000,4,Pub,Hotel,Gastropub,Italian Restaurant,Coffee Shop,Park,Café,Brewery,French Restaurant,Canal Lock
1,Cambridgeshire,9.02,0.083333,52.333333,0,Pub,Café,Bar,Coffee Shop,Burger Joint,Park,Gastropub,Restaurant,Bookstore,Tea Room
2,Cumbria,5.62,-2.942098,54.614314,0,Pub,Hotel,Café,Restaurant,Bar,Coffee Shop,Gastropub,Supermarket,Scenic Lookout,Italian Restaurant
3,Derbyshire,6.04,-1.556618,53.118503,2,Pub,Park,Coffee Shop,Theme Park Ride / Attraction,Café,Scenic Lookout,Historic Site,Theater,Gastropub,Food & Drink Shop
4,Devon,9.42,-3.750000,50.750000,0,Pub,Café,Hotel,Bar,Gastropub,Coffee Shop,Beach,Italian Restaurant,Trail,Restaurant


### Now we can visualise the clusters


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(county_merged['Latitude'], county_merged['Longitude'], county_merged['County'], county_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters